# Logistic Data Fitting

Notebook for experimenting with different models to fit Quai data.

## Inputs

1. $\beta_0$ - Intercept of logistic regression
2. $\beta_1$ - Coeffecient of difficulty over $log_2$ difficulty
3. $d$ - The current block difficulty

## Probability of Taking Qi

$$\frac{1}{1+exp(-(\beta_0 + \beta_1 \cdot \frac{d}{log_2(d)}))}$$

## Equal Likelihood Probability Parameters

$$\frac{1}{1+exp(-(\beta_0 + \beta_1 \cdot \frac{d}{log_2(d)}))} = .5$$

$$.5 = .5 \cdot [exp(-(\beta_0 + \beta_1 \cdot \frac{d}{log_2(d)}))]$$

$$1 = exp(-(\beta_0 + \beta_1 \cdot \frac{d}{log_2(d)}))$$

$$0 = \beta_0 + \beta_1 \cdot \frac{d}{log_2(d)}$$

$$\beta_0 =  - \beta_1 \cdot \frac{d}{log_2(d)}$$

## Synthetic Data

In [5]:
from math import log2


##### Inputs

# Difficulty where p = .5
midpoint_difficulty = 1000

# Left and right bounds for difficulty
left_difficulty_endpoint = 200
right_difficulty_endpoint = 2000

# Any value > 0 for beta1 parameter
beta1 = .001

##### Calculations
beta0 = midpoint_difficulty / log2(midpoint_difficulty) * - beta1

-0.10034333188799373


## Simulation Data

In [ ]:
from pandas.core.common import flatten
from math import log2

Y = list(flatten([x["Qi Taken"] for x in state["Mining Log"]]))
Y = [x>0 for x in Y]
Y = np.array(Y).astype(int)
X = list(flatten([x["Block Difficulty"] for x in state["Mining Log"]]))
X = [x/log2(x) for x in X]
X = [[1,x] for x in X]
X = np.array(X)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='none', fit_intercept=False
    )
model.fit(X[:4000], Y[:4000])
print(model.coef_)

model = LogisticRegression(penalty='none', fit_intercept=False
    )
model.fit(X[4000:8000], Y[4000:8000])
print(model.coef_)

model = LogisticRegression(penalty='none', fit_intercept=False
    )
model.fit(X[8000:12000], Y[8000:12000])
print(model.coef_)

model = LogisticRegression(penalty='none', fit_intercept=False
    )
model.fit(X[12000:16000], Y[12000:16000])
print(model.coef_)

model = LogisticRegression(penalty='none', fit_intercept=False
    )
model.fit(X, Y)
print(model.coef_)

from sklearn.metrics import r2_score


print(model.score(X, Y))
print(Y.mean())
print(np.corrcoef(X[:,1], Y))
print(model.predict(X))
print(r2_score(Y, model.predict(X)))